In [1]:
import os
from pyspark.sql import SparkSession
from pyspark.ml import PipelineModel
from pyspark.sql.types import StructType, StructField, StringType, FloatType, IntegerType
from pyspark.sql.functions import col, length, when, log1p, expm1, lower, lit

spark = SparkSession.builder \
    .appName("Inference LR") \
    .config("spark.driver.memory", "4g") \
    .master("local[*]") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

MODEL_PATH = "../../models/regression/lr_price_log_v1"

print(f"Loading Linear Regression model from {MODEL_PATH}...")
try:
    model = PipelineModel.load(MODEL_PATH)
    print("Model loaded successfully!")
except Exception as e:
    print(f"Error loading model: {e}")
    model = None

Loading Linear Regression model from ../../models/regression/lr_price_log_v1...
Model loaded successfully!


In [4]:
import math 

def predict_price_lr(title, category, rating, rating_count, store_freq=100):
    if model is None:
        return "Model not loaded"

    schema = StructType([
        StructField("title", StringType(), True),
        StructField("main_category", StringType(), True),
        StructField("average_rating", FloatType(), True),
        StructField("rating_number", IntegerType(), True),
        StructField("store_freq", IntegerType(), True),
        StructField("features_count", IntegerType(), True)
    ])
    
    data = [(title, category, float(rating), int(rating_count), int(store_freq), 0)]
    input_df = spark.createDataFrame(data, schema=schema)
    
    input_df = input_df.withColumn("title_len", length(col("title")))
    input_df = input_df.withColumn("is_premium", when(lower(col("title")).rlike("premium|pro|deluxe"), 1).otherwise(0))
    input_df = input_df.withColumn("is_bundle", when(lower(col("title")).rlike("bundle|set|pack"), 1).otherwise(0))
    input_df = input_df.withColumn("log_rating_number", log1p(col("rating_number")))
    input_df = input_df.withColumn("log_store_freq", log1p(col("store_freq")))
    
    prediction_df = model.transform(input_df)
    
    predicted_log_price = prediction_df.select("prediction").collect()[0][0]
    
    real_price = math.expm1(predicted_log_price)
    
    return max(0.0, real_price)

In [5]:
item = {
    "title": "Samsung Galaxy S24 Ultra Case Premium",
    "category": "Cell Phones & Accessories",
    "rating": 4.8,
    "rating_count": 1500,
    "store_freq": 500
}

price = predict_price_lr(**item)

print(f"Product: {item['title']}")
print(f"Predicted Price (Linear Regression): ${price:.2f}")

Product: Samsung Galaxy S24 Ultra Case Premium
Predicted Price (Linear Regression): $22.09
